In [2]:
#Libraries for data manipulation
import pandas as pd
import numpy as np

#Library for splitting data
from sklearn.model_selection import train_test_split 

#Library for hyperparameter tuning
from sklearn.model_selection import GridSearchCV

In [ ]:
#Dictionary with the best hyperparameters for each model
#hyperparameters = {
    #'randomForest' : ,
    #'xgboost' : ,
    #'decissionTreClassifier' : ,
    #'knn' : ,
    #'svm': ,
    #'lgb' : ,
#}

In [ ]:

#Function thar returns the best hyperparameters for each model
#def getHyperparameters(model):
    #return hyperparameters['model']

In [3]:
trainCSV = pd.read_csv('./forHiper')
trainCSV.head()

,id,keyword,location,text,target,users_tagged,hashtags,links,tweet_length,tweet_size,...,users_tagged_minMaxNorm,hashtags_minMaxNorm,hashtags_natNat,users_tagged_natNat,links_natNat,Porn_words_natNat,links_hash_natNat,links_users_natNat,hash_users_natNat,norm2
0,1,earthquake,us,deed reason earthquake may allah forgive u,1,0,1,0,69,5,...,0.0,0.076923,0.142857,0.0,0.0,0.0,0.142857,0.0,0.142857,20.505016
1,4,fire,la,forest fire near la ronge sask canada,1,0,0,0,38,0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,15.093579
2,5,evacuation,unknown,resident asked shelter place notified oficers ...,1,0,0,0,133,10,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,30.676623
3,6,evacuation,california,people receive wildfire evacuation order calif...,1,0,1,0,65,5,...,0.0,0.076923,0.142857,0.0,0.0,0.0,0.142857,0.0,0.142857,20.790918
4,7,smoke,alaska,got sent photo ruby alaska smoke wildfire pour...,1,0,2,0,88,5,...,0.0,0.153846,0.222222,0.0,0.0,0.0,0.222222,0.0,0.222222,28.465530


In [ ]:
trainCSV = trainCSV.iloc[:, 4:]
x, y = trainCSV.iloc[:,1:], trainCSV.iloc[:,0]  #X tiene que tener todos los features distintos al target
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [ ]:
#Searchs for the best hyperparameters
def findHyperparameters(xTrain, yTrain, model, param_grid, cv, vb, nj):
    
    grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = 'roc_auc', cv = cv, verbose = vb, n_jobs = nj)
    grid.fit(xTrain, yTrain)
    return grid.best_params_

## Random forest

https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators' : [100, 300, 500, 800, 1200],
    'max_depth' : [5, 8, 15, 25, 30],
    'min_samples_split' : [2, 5, 10, 15, 100],
    'min_samples_leaf' : [1, 2, 5, 10],
    'criterion' : ['gini','entropy']
}

RC = RandomForestClassifier()
bestParametersRF = findHyperparameters(xTrain, yTrain, RC, param_grid, 3, 1, -1) #bestParametersRF
bestParametersRF

## XGBoost :)

https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
folds = 3
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
}
XGB = XGBClassifier()
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
bestParametersXGB = findHyperparameters(xTrain, yTrain, XGB, param_grid, skf.split(xTrain, yTrain), 3, 4)
bestParametersXGB

## Decission Tree Classifier

https://www.kaggle.com/mayu0116/hyper-parameters-tuning-of-dtree-rf-svm-knn

In [ ]:
from sklearn import tree

param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'random_state':[123]}

DTC = tree.DecisionTreeClassifier(min_samples_split = 10)
bestParameteresDTC = findHyperparameters(xTrain, yTrain, DTC, param_grid, 3, 1, -1)
bestParameteresDTC

## KNN

https://www.kaggle.com/mayu0116/hyper-parameters-tuning-of-dtree-rf-svm-knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors':[5,6,7,8,9,10],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}

KNN = KNeighborsClassifier()
bestParameteresKNN = findHyperparameters(xTrain, yTrain, KNN, param_grid, 3, 1, 1)
bestParameteresKNN

## Support Vector Machine

https://www.kaggle.com/mayu0116/hyper-parameters-tuning-of-dtree-rf-svm-knn

In [ ]:
from sklearn import svm

param_grid = {'C': [6,7,8,9,10,11,12], 
          'kernel': ['linear','rbf']}

SVC = svm.SVC()
bestParameteresSVC = findHyperparameters(xTrain, yTrain, SVC, param_grid, 3, 1, -1)
bestParameteresSVC

## Light GBM

https://mlfromscratch.com/gridsearch-keras-sklearn/#/

In [ ]:
import lightgbm as lgb

param_grid = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}

LGBM = lgb.LGBMClassifier()

bestParameteresLGBM = findHyperparameters(xTrain, yTrain, LGBM, param_grid, 5, 2, -1)
bestParameteresLGBM

## MLPClassifier

https://medium.com/@panjeh/scikit-learn-hyperparameter-optimization-for-mlpclassifier-4d670413042b

In [ ]:
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

MLP = MLPClassifier(max_iter=100)

bestParameteresMLP = findHyperparameters(xTrain, yTrain, MLP, param_grid, 5, 2, -1)
bestParameteresMLP